### 설정 + 필요한 모델 다운로드

In [1]:
# !pip install konlpy

In [2]:
# !pip install transformers

In [3]:
# !pip install git+https://github.com/SKT-AI/KoBART #egg=kobart

In [1]:
import torch
import os 
import sys
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()
# import wandb
os.environ["WANDB_DISABLED"] = "true"

# from google.colab import drive
import re
from konlpy.tag import Okt
import requests
from bs4 import BeautifulSoup

from itertools import combinations
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import SGD
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from torch.utils.data import Dataset, DataLoader,random_split

from kobart import get_pytorch_kobart_model, get_kobart_tokenizer
from transformers import BartModel

In [2]:
# from transformers import AutoModel

In [3]:
# seed
seed = 7777
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
# device type
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"# available GPUs : {torch.cuda.device_count()}")
    print(f"GPU name : {torch.cuda.get_device_name()}")
else:
    device = torch.device("cpu")
print(device)

# available GPUs : 1
GPU name : Quadro RTX 6000
cuda


### 원본 데이터 불러오기

In [4]:
df = pd.read_csv("../sports_news_data.csv")

In [5]:
df.head(10)

,TITLE,CONTENT,PUBLISH_DT
0,스털링 다이빙 논란 종결?… “오른쪽 다리 접촉 있었잖아”,[스포탈코리아] 유럽축구연맹(UEFA) 유로 2020 심판위원장 로베르토 로세티가 ...,2021-07-15
1,"‘디 마리아 없다’ 유로X코파 베스트11, 이탈리아만 7명",[스포탈코리아] 유로 2020과 코파 아메리카 2021로 베스트11을 만든다면 어떤...,2021-07-15
2,‘슈퍼컴퓨터 예측’ 맨시티 우승-맨유 4위… 토트넘은 ‘6위’,[스포탈코리아] 새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다.\n\...,2021-07-15
3,"“이재성, 완벽한 프로… 마인츠서 성공할 것” 킬 디렉터의 애정 듬뿍 응원",[스포탈코리아] 홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠...,2021-07-15
4,"‘홈킷과 딴판’ 바르사 팬들, NEW 어웨이 셔츠 호평… “가장 좋아하는 색!”",[스포탈코리아] FC 바르셀로나가 새 시즌 원정 유니폼을 공개했다. 팬들은 만족스럽...,2021-07-15
5,"긴급 수혈된 바르사 NO.9, 1년 반 만에 떠난다… ‘EPL행 유력’",[스포탈코리아] FC 바르셀로나는 새 시즌을 앞두고 선수단 정리가 한창이다. 잉여 ...,2021-07-15
6,"[김남구의 유럽통신] 황의조, 손흥민 소속사와 손잡다… CAA Base와 계약",[스포탈코리아=파리(프랑스)] 황의조(지롱댕 드 보르도)가 한국 선수로는 3번째로 ...,2021-07-15
7,"""메시 종신은 축복!""…스폰서 5년 더 보장, 바르셀로나 함박웃음",[스포탈코리아] 리오넬 메시(34)가 FC바르셀로나에 남는다. 연봉을 절반 삭감하지...,2021-07-15
8,"[오피셜] 눈물 흘렸던 '37세 전설' 로번, 두 번째 현역 은퇴 발표",[스포탈코리아] 네덜란드 축구스타 아르연 로번(37)이 현역 은퇴를 밝혔다. \n\...,2021-07-15
9,"'100세' 메시팬 할아버지, 748골 수기 작성…메시도 감사 인사",[스포탈코리아] 리오넬 메시(34)는 프로 데뷔하고 748골을 터뜨렸다. 전산화 하...,2021-07-15


In [6]:
df[df['CONTENT'].isnull() == True]

,TITLE,CONTENT,PUBLISH_DT
4137,GOAL50 2021 투표하기,NaN,2021-11-02
5475,"[GOAL LIVE] '오직 익수' 안익수가 생각하는 팬의 의미 ""상당히 두려운 존재""",NaN,2021-12-04


In [7]:
# 결측치 확인
df.isnull().sum()

TITLE         0
CONTENT       2
PUBLISH_DT    0
dtype: int64

In [8]:
# 결측치 제거
df = df.dropna()
df.isnull().sum()

TITLE         0
CONTENT       0
PUBLISH_DT    0
dtype: int64

In [9]:
# 중복값 제거(CONTENT만 처리하므로 TITLE은 두기로 한다.)
idx = df['CONTENT'].drop_duplicates().index
df = df.loc[idx]
df.reset_index(drop=True, inplace = True)
print(df.shape) ; df.head(5)

(9050, 3)


,TITLE,CONTENT,PUBLISH_DT
0,스털링 다이빙 논란 종결?… “오른쪽 다리 접촉 있었잖아”,[스포탈코리아] 유럽축구연맹(UEFA) 유로 2020 심판위원장 로베르토 로세티가 ...,2021-07-15
1,"‘디 마리아 없다’ 유로X코파 베스트11, 이탈리아만 7명",[스포탈코리아] 유로 2020과 코파 아메리카 2021로 베스트11을 만든다면 어떤...,2021-07-15
2,‘슈퍼컴퓨터 예측’ 맨시티 우승-맨유 4위… 토트넘은 ‘6위’,[스포탈코리아] 새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다.\n\...,2021-07-15
3,"“이재성, 완벽한 프로… 마인츠서 성공할 것” 킬 디렉터의 애정 듬뿍 응원",[스포탈코리아] 홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠...,2021-07-15
4,"‘홈킷과 딴판’ 바르사 팬들, NEW 어웨이 셔츠 호평… “가장 좋아하는 색!”",[스포탈코리아] FC 바르셀로나가 새 시즌 원정 유니폼을 공개했다. 팬들은 만족스럽...,2021-07-15


### 전처리
- 중복 및 결측치 제거
- 크롤링 상에서 생긴 쓸모 없는 문구 처리


In [11]:
# 타이틀
def title_cleansing(x):
    new_string = re.sub(r'\([^)]*\)|\[[^)]*\]|\<[^)]*\>', '', x)
    return new_string

# 본문
def content_cleansing(string):
    try:
        if '스포탈코리아' in string:
            cleanr =re.compile('<.*?>')
            cleantext = re.sub(cleanr, '', string).replace("&nbsp;", "").replace('\n',"").replace('\t',"").replace('\xa0', "")


            new_string = re.sub(r'사진=.+$', '', cleantext)
            new_string = re.sub(r'.+.기자=', '', new_string)    
            new_string = re.sub(r'\([^)]*\)|\[[^)]*\]|\<[^)]*\>', '', new_string)
            new_string = re.sub(r"[^가-힣a-zA-Z0-9一-龥. ]","",new_string)

            return new_string.strip()

        elif '골닷컴' in string:   ### +++ 기자 이름
            string = re.sub(r'(?<=\<hr style="color: #00ac77; width: 100%; background: #00ac77; border: 0; height: 2px;" /><span><strong>)(.*?)(?=<\/span>)', '', string)

            cleanr =re.compile('<.*?>')
            cleantext = re.sub(cleanr, '', string).replace("&nbsp;", "").replace('\n',"").replace('\t',"").replace('\xa0', "")

            new_string = re.sub(r'\([^)]*\)|\[[^)]*\]|\<[^)]*\>', '', cleantext)
            new_string = re.sub(r"[^가-힣a-zA-Z0-9一-龥. ]","", new_string)

            return new_string.strip()

        else:
            cleanr =re.compile('<.*?>')
            cleantext = re.sub(cleanr, '', string).replace("&nbsp;", "").replace('\n',"").replace('\t',"").replace('\xa0', "")

            new_string = re.sub(r'\([^)]*\)|\[[^)]*\]|\<[^)]*\>', '', cleantext)
            new_string = re.sub(r"[^가-힣a-zA-Z0-9一-龥. ]","", new_string)
            
            return new_string.strip()
        
    except:
        return "문제"

In [12]:
df['cleaned_TITLE'] = df['TITLE'].progress_apply(lambda x: title_cleansing(x))
df['cleaned_CONTENT'] = df['CONTENT'].progress_apply(lambda x: content_cleansing(x))

100%|██████████| 9050/9050 [00:27<00:00, 333.67it/s]


In [13]:
df['cleaned_TITLE'].value_counts()

'OT 14경기 연속 실점' 맨유, 63년 만에 불명예 기록         2
 '감독 대행' 지냈던 메이슨, 콘테 사단 합류                2
콘테의 약속 "팬들 위해 매력적 축구 선보이겠다"               2
울버햄튼, 황희찬 완전 영입 자신…"우리 축구에 필요해"           2
'힘든 시기' 반 더 비크에게 찾아온 새로운 축복…'예비 아빠' 됐다    2
                                         ..
“손흥민은 토트넘 레전드, 아시아에서 누구도 못 넘어” 외신 극찬      1
아직도 '메시 유니폼' 파는 바르셀로나, 100만원↑ 고가 판매 수익    1
"황의조, 문전 집중력 좋아졌다" 상승한 슈팅 정확도 조명          1
콘테의 한숨 "로메로 부상 정말 심각해. 내년 2월에 볼 수도.."     1
'메시vs호날두→비니시우스vs뎀벨레' 확 달라질 엘 클라시코?        1
Name: cleaned_TITLE, Length: 8995, dtype: int64

In [14]:
df['cleaned_CONTENT'].value_counts()

배시온 기자 울버햄튼은 에버튼과 1일 몰리뉴 스타디움에서 202122시즌 프리미어리그 10라운드 경기를 치렀다. 울버햄튼은 킬먼 히메네스 득점에 힘입어 후반전 이워비에게 실점을 허용했음에도 21 승리를 거뒀다. 울버햄튼은 5경기 연속 무패를 이어가며 리그 7위로 도약했다.이날 황희찬은 선발 출전해 후반 추가시간 교체까지 약 90분간 그라운드를 누볐다. 특히 라울 히메네스와 뛰어난 연계 플레이를 보이며 전반전 울버햄튼 공격을 활발히 하는데 힘썼다.전반 14분 황희찬은 선제골을 터트리는 듯 했으나 이내 아쉬운 상황을 맞이했다. 히메네스가 에버튼 수비진 사이를 돌파해 황희찬에게 연결했고 이를 받은 황희찬의 슈팅이 에버튼 골망을 흔들었으나 VAR 판독 후 오프사이드 판정이 선언된 것. 결국 황희찬의 득점은 취소되며 리그 5호골 역시 다음으로 미뤄졌다.그럼에도 황희찬은 활발한 공격 찬스를 만들었다. 전반 13분 황희찬의 패스는 트린캉에게까지 이어졌고 날카로운 슈팅이 나왔으나 픽포드의 선방으로 득점엔 실패했다. 후반전에 들어서도 황희찬은 측면 돌파 후 트린캉에게 패스하며 기회를 살폈지만 아쉽게 한 점을 더 달아나진 못했다.경기 후 축구통계매체 후스코어드닷컴은 황희찬에게 평점 6.9점을 부여했다. 팀 내 최고점은 선제골을 기록한 막시밀리안 킬먼으로 평점 8.4점을 받았다. 추가골의 주인공 라울 히메네스가 8점 트린캉과 리얀 아이트 누리가 7.2점으로 그 뒤를 이었다.                                                                                                                                                                                                                                                                                                

In [16]:
# 데이터 전처리 후 중복값 제거
idx = df['cleaned_CONTENT'].drop_duplicates().index
df = df.loc[idx]
df.reset_index(drop=True, inplace = True)
print(df.shape) ; df.head(5)

(8998, 5)


,TITLE,CONTENT,PUBLISH_DT,cleaned_TITLE,cleaned_CONTENT
0,스털링 다이빙 논란 종결?… “오른쪽 다리 접촉 있었잖아”,[스포탈코리아] 유럽축구연맹(UEFA) 유로 2020 심판위원장 로베르토 로세티가 ...,2021-07-15,스털링 다이빙 논란 종결?… “오른쪽 다리 접촉 있었잖아”,유럽축구연맹 유로 2020 심판위원장 로베르토 로세티가 잉글랜드와 덴마크전에 나온 ...
1,"‘디 마리아 없다’ 유로X코파 베스트11, 이탈리아만 7명",[스포탈코리아] 유로 2020과 코파 아메리카 2021로 베스트11을 만든다면 어떤...,2021-07-15,"‘디 마리아 없다’ 유로X코파 베스트11, 이탈리아만 7명",유로 2020과 코파 아메리카 2021로 베스트11을 만든다면 어떤 모습일까.지난달...
2,‘슈퍼컴퓨터 예측’ 맨시티 우승-맨유 4위… 토트넘은 ‘6위’,[스포탈코리아] 새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다.\n\...,2021-07-15,‘슈퍼컴퓨터 예측’ 맨시티 우승-맨유 4위… 토트넘은 ‘6위’,새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다.영국 매체 스포츠 바이...
3,"“이재성, 완벽한 프로… 마인츠서 성공할 것” 킬 디렉터의 애정 듬뿍 응원",[스포탈코리아] 홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠...,2021-07-15,"“이재성, 완벽한 프로… 마인츠서 성공할 것” 킬 디렉터의 애정 듬뿍 응원",홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠다.이재성은 20...
4,"‘홈킷과 딴판’ 바르사 팬들, NEW 어웨이 셔츠 호평… “가장 좋아하는 색!”",[스포탈코리아] FC 바르셀로나가 새 시즌 원정 유니폼을 공개했다. 팬들은 만족스럽...,2021-07-15,"‘홈킷과 딴판’ 바르사 팬들, NEW 어웨이 셔츠 호평… “가장 좋아하는 색!”",FC 바르셀로나가 새 시즌 원정 유니폼을 공개했다. 팬들은 만족스럽다는 반응이다.바...


In [17]:
df['cleaned_CONTENT'].value_counts()

과거 맨체스터 유나이티드에서 뛰었던 루크 채드윅이 잭 그릴리쉬를 리오넬 메시와 비교하며 극찬했다.영국 매체 맨체스터이브닝 뉴스는 30일 채드윅이 그릴리쉬를 세계 최고의 선수인 메시와 비교했다라고 전했다.그릴리쉬는 잉글리시 프리미어리그에서 손꼽히는 플레이메이커다. 202021시즌 EPL 26경기에 출전해 6골 10도움을 기록하며 빌라의 공격을 이끌었다. 자연스레 여러 빅클럽의 관심이 집중됐고 맨시티가 가장 적극적으로 영입에 나섰다.맨시티는 올 여름 이적시장에서 최우선 영입 대상으로 그릴리쉬를 낙점했다. 해리 케인보다 더 우선이라는 보도도 나왔다. 하지만 빌라는 그릴리쉬의 잔류를 원했고 맨시티에 천문학적인 이적료를 요구했다. 올 여름 가장 핫한 매물로 떠오른 그릴리쉬다.채드윅은 왜 그릴리쉬가 최고의 매물로 떠오른 지에 대한 이유를 설명했다. 그는 커트오브사이드와 인터뷰를 통해 그릴리쉬는 이미 월드클래스거나 월드클래스가 될 준비를 마쳤다고 생각한다라며 엄지를 치켜세웠다.이어 그릴리쉬의 플레이 스타일은 맨시티에 매우 잘 어울린다. 메시는 펩 과르디올라 감독 체제일 때 빠른 드리블을 통해 상대 수비수들을 제치고 기회를 창출하는 역할을 했다. 그릴리쉬가 메시와 비슷한 선수라고 본다. 메시와 동급은 아니지만 유사하다라고 강조했다.                                                                                                                                                                                                                                                                                                                                                                              

In [ ]:
idx_list=[]
for i in range(len(trimmed_data)):
  if len(trimmed_data['CONTENT'].iloc[i]) < 4:
    idx_list.append(trimmed_data.iloc[i].name)

abandoned_data = trimmed_data.loc[idx_list]
trimmed_data = trimmed_data.drop(idx_list)

In [ ]:
print(trimmed_data['CONTENT'].loc[3189])

In [ ]:
len(trimmed_data)

In [ ]:
trimmed_data['CONTENT'].iloc[8991]

- 띄어쓰기 및 불용어 처리

In [ ]:
# 한국어 불용어 리스트 크롤링


url = "https://www.ranks.nl/stopwords/korean"
response = requests.get(url, verify = False)

if response.status_code == 200:
    soup = BeautifulSoup(response.text,'html.parser')
    content = soup.select_one('#article178ebefbfb1b165454ec9f168f545239 > div.panel-body > table > tbody > tr')
    stop_words=[]
    for x in content.strings:
        x=x.strip()
        if x:
            stop_words.append(x)
    print(f"# Korean stop words: {len(stop_words)}")
else:
    print(response.status_code)

In [ ]:

okt = Okt()
for i in tqdm(range(len(trimmed_data))):
  temp_data = okt.morphs(trimmed_data["TITLE"].iloc[i])
  temp_list = []

  for word in temp_data:
    if word in stop_words: continue
    temp_list.append(word)
  
  trimmed_data["TITLE"].iloc[i] = " ".join(temp_list)

  temp_list = []
  for sentence in trimmed_data["CONTENT"].iloc[i]:
    temp_data = okt.morphs(sentence)
    temp_sentecne_list = []
    
    for word in temp_data:
      if word in stop_words: continue
      temp_sentecne_list.append(word)
    
    temp_sentence = " ".join(temp_sentecne_list)
    temp_list.append(temp_sentence)
  
  trimmed_data["CONTENT"].iloc[i] = temp_list
  


In [ ]:
trimmed_data.head()

In [ ]:
data_row = trimmed_data.iloc[0]
print(data_row)

text = data_row['CONTENT']
print(text)

for i, sentence in enumerate(text):
  print(i)
  print(sentence)

In [ ]:
max_sentence_num = 0
for i in range(len(trimmed_data)):
  sentence_num = len(trimmed_data["CONTENT"].iloc[i])
  max_sentence_num = max(sentence_num,max_sentence_num)

print(max_sentence_num)

#Extractive summarization - Matchsum

### Dataset & Dataloader 생성

In [ ]:
def control_input_ids(input_ids_tensor,length,cls_token_num,sep_token_num,pad_token_num):
  cur_length = len(input_ids_tensor)
  cls_token = torch.tensor([cls_token_num])
  sep_token = torch.tensor([sep_token_num])

  if cur_length+2 > length:
    input_ids_tensor = input_ids_tensor[:length-2]  # 길이가 넘치면 자른다
    return torch.cat([cls_token,input_ids_tensor,sep_token])
  else:
    input_ids_tensor = torch.cat([cls_token,input_ids_tensor,sep_token])
    padding_list = torch.tensor([pad_token_num]*(length - cur_length -2)) # 길이가 모자라면 padding token 을 채운다
    return torch.cat([input_ids_tensor,padding_list])

In [ ]:
def custom_collate_fn(samples):
  
  text_ids = torch.empty(0,512)
  labels_ids = torch.empty(0,32)
  for sample in samples:
    text_ids = torch.cat([text_ids,sample['text_input_ids'].unsqueeze(0)],dim=0) 
    labels_ids = torch.cat([labels_ids,sample['labels_input_ids'].unsqueeze(0)],dim=0)

  sentence_input_ids = [sample['sentence_input_ids'] for sample in samples]
  nn.utils.rnn.pad_sequence(sentence_input_ids,batch_first=True,padding_value = 1)

  return dict(text_input_ids = text_ids.to(torch.int64), labels_input_ids = labels_ids.to(torch.int64), sentence_input_ids = sentence_input_ids)

In [ ]:
class CustomDataset(Dataset):
  def __init__(
      self, data, tokenizer,
      text_max_token_len = 512,
      summary_max_token_len = 32
        ):
    self.tokenizer = tokenizer
    self.data = data
    self.text_max_token_len = text_max_token_len
    self.summary_max_token_len = summary_max_token_len
  def __len__(self):
    return len(self.data)
  
  def __getitem__(self, index):
    cls_token_num = 0
    sep_token_num = 2
    pad_token_num = 1
    
    data_row = self.data.iloc[index]
    text = data_row['CONTENT']
    
    total_text_ids = torch.tensor([])
    sentence_input_ids = torch.empty(0,32)

    for sentence in text:
      text_encoding_sentence = self.tokenizer(
          sentence,return_tensors = "pt",add_special_tokens=False)
      sentence_indiv_input_ids = text_encoding_sentence['input_ids'].flatten()
      total_text_ids = torch.cat([total_text_ids,sentence_indiv_input_ids])

      sentence_indiv_input_ids = control_input_ids(sentence_indiv_input_ids,self.summary_max_token_len,cls_token_num,sep_token_num,pad_token_num)
      sentence_indiv_input_ids = sentence_indiv_input_ids.unsqueeze(0)
      sentence_input_ids = torch.cat([sentence_input_ids,sentence_indiv_input_ids],dim=0)
    
    sentence_input_ids = sentence_input_ids.to(torch.int64)
    total_text_ids = control_input_ids(total_text_ids,self.text_max_token_len,cls_token_num,sep_token_num,pad_token_num)    
    total_text_ids = total_text_ids

    labels = data_row['TITLE']
    summary_encoding = self.tokenizer(
        labels,
        add_special_tokens = False,
        return_tensors = "pt"
    )

    labels_ids = summary_encoding['input_ids'].flatten()
    labels_ids = control_input_ids(labels_ids,self.summary_max_token_len,cls_token_num,sep_token_num,pad_token_num)

    return dict(text_input_ids = total_text_ids, labels_input_ids = labels_ids, sentence_input_ids = sentence_input_ids)


In [ ]:
tokenizer = get_kobart_tokenizer()

In [ ]:
whole_dataset = CustomDataset(trimmed_data,tokenizer)

train_set_num = 7000
train_dataset , valid_dataset = random_split(whole_dataset, [train_set_num,len(trimmed_data)-train_set_num])
train_dataloader = DataLoader(train_dataset, batch_size = 2, shuffle=True,collate_fn = custom_collate_fn)
valid_dataloader =  DataLoader(valid_dataset, batch_size = 2, shuffle=False,collate_fn = custom_collate_fn)

### Matchsum

- 평가 metric -> rdass
- 기본적으로 모델에 스코어가 높은 5개의 단일 문장을 뽑고 뽑인 문장으로 만들어진 조합 가운데서 스코어가 높은 조합을 golden summary로 선정
- loss 는 margin ranking loss 사용


In [ ]:
def get_score(doc,label,answer):
  score_1 = torch.cosine_similarity(doc,answer,dim=0)
  score_2 = torch.cosine_similarity(label,answer,dim=0)
  return score_1+score_2

In [ ]:
def get_candidate_id(doc_emb,summary_emb,batch_sentence_id, candidate_num, extract_model,device):
    cls_token = torch.tensor([0]).to(device)
    sep_token = torch.tensor([2]).to(device)
    candidate_ids = torch.empty([0,candidate_num,128]).to(device)
    
    for batch_idx, sentence_id_tensor in enumerate(batch_sentence_id):
      sentence_id_tensor = sentence_id_tensor.to(device)
      out = extract_model.forward(sentence_id_tensor)  #sentence_id_tensor = [문장 갯수,32개의 토큰]
      hidden_states = out['last_hidden_state'][:,0,:] # [문장 갯수,token 갯수 ,768 dim_vec]
      score_list= []
      
      for i in range(hidden_states.shape[0]):
        score = get_score(doc = doc_emb[batch_idx,:], label = summary_emb[batch_idx,:], answer = hidden_states[i,:])
        score_list.append((score,i))
      
      score_list.sort(key = lambda x: x[0],reverse=True)
      idx_list = [idx for _,idx in score_list][:5]
    
      # get candidate summaries
      # here is for CNN/DM: truncate each document into the 5 most important sentences (using BertExt), 
      # then select any 2 or 3 sentences to form a candidate summary, so there are C(5,2)+C(5,3)=20 candidate summaries.
      # if you want to process other datasets, you may need to adjust these numbers according to specific situation.
      indices = list(combinations(idx_list, 2))
      indices += list(combinations(idx_list, 3))
      if len(idx_list) < 2:
          indices = [idx_list]
    
      # get score for each candidate summary and sort them in descending order
      score = []
      for i in indices:
          i = list(i)
          i.sort()
          # write dec
          dec = torch.tensor([]).to(device)
          for j in i:
              sent = sentence_id_tensor[j]
              sent = sent[1:]
              sep_token_idx = 0
              for token_idx in range(len(sent)):
                if sent[token_idx] == 2: break
                else:sep_token_idx += 1
              sent = sent[:sep_token_idx]
              dec = torch.cat([dec,sent],dim=0)
          
          dec = torch.cat([cls_token,dec,sep_token],dim=0)
          dec = dec.to(torch.int64)
          dec_out = extract_model.forward(input_ids = dec.unsqueeze(0))
          score.append((dec, get_score(doc_emb[batch_idx,:],summary_emb[batch_idx,:], dec_out['last_hidden_state'][0,0,:])))
      
      score.sort(key=lambda x : x[1], reverse=True)
      score = score[:candidate_num]
      
      candidate_ids_ind= torch.empty(0,128).to(device)
      for k,_ in score:
        dec = k
        if len(dec) < 128:
          padding_list = torch.tensor([1]*(128-len(dec))).to(device)
          dec = torch.cat([k,padding_list],dim=0)
        else:
          dec = dec[:128]

        candidate_ids_ind = torch.cat([candidate_ids_ind,dec.unsqueeze(0)],dim = 0)

      candidate_ids = torch.cat([candidate_ids,candidate_ids_ind.unsqueeze(0)],dim = 0)

    return candidate_ids.to(torch.int64)

In [ ]:
class MatchSum(nn.Module):  
    def __init__ (self, encoder, candidate_num, device,hidden_size=768):
        super(MatchSum, self).__init__()
        
        self.hidden_size = hidden_size
        self.candidate_num  = candidate_num
        self.encoder = encoder
        self.device = device

    def forward(self, text_id, summary_id,list_of_sentence_id):
        
        batch_size = text_id.size(0)
        pad_id = 1 

        # get document embedding
        input_mask = ~(text_id == pad_id)
        out = self.encoder(text_id, attention_mask=input_mask)['last_hidden_state'] # last layer
        doc_emb = out[:, 0, :]
        assert doc_emb.size() == (batch_size, self.hidden_size) # [batch_size, hidden_size]
        
        # get summary embedding
        input_mask = ~(summary_id == pad_id)
        out = self.encoder(summary_id, attention_mask=input_mask)['last_hidden_state'] # last layer
        summary_emb = out[:, 0, :]
        assert summary_emb.size() == (batch_size, self.hidden_size) # [batch_size, hidden_size]

        # get summary score
        summary_score = torch.cosine_similarity(summary_emb, doc_emb, dim=-1)

        # get candidate embedding
        candidate_id = get_candidate_id(doc_emb,summary_emb,list_of_sentence_id, self.candidate_num, self.encoder,self.device) #[batch_size , candidate_num, token_num]
        candidate_id = candidate_id.view(-1, candidate_id.size(-1)) 
        input_mask = ~(candidate_id == pad_id)
        out = self.encoder(candidate_id, attention_mask=input_mask)['last_hidden_state'] 
        candidate_emb = out[:, 0, :].view(batch_size, self.candidate_num, self.hidden_size)  # [batch_size, candidate_num, hidden_size]
        assert candidate_emb.size() == (batch_size, self.candidate_num, self.hidden_size)
        
        # get candidate score
        doc_emb = doc_emb.unsqueeze(1).expand_as(candidate_emb)
        score = torch.cosine_similarity(candidate_emb, doc_emb, dim=-1) # [batch_size, candidate_num]
        golden_list = torch.argmax(score,dim=1)
        assert score.size() == (batch_size, self.candidate_num)

        return {'score': score, 'summary_score': summary_score, 
                'golden_summary':torch.cat([candidate_id[0,:].unsqueeze(0),candidate_id[self.candidate_num,:].unsqueeze(1)],dim=1)}

In [ ]:
class MarginRankingLoss():      
    
    def __init__(self, margin, score=None, summary_score=None):
        super(MarginRankingLoss, self).__init__()
        # self._init_param_map(score=score, summary_score=summary_score)
        self.margin = margin
        self.loss_func = torch.nn.MarginRankingLoss(margin)

    def get_loss(self, score, summary_score):
        
        # equivalent to initializing TotalLoss to 0
        # here is to avoid that some special samples will not go into the following for loop
        ones = torch.ones(score.size()).cuda(score.device)
        loss_func = torch.nn.MarginRankingLoss(0.0)
        TotalLoss = loss_func(score, score, ones)

        # candidate loss
        n = score.size(1)
        for i in range(1, n):
            pos_score = score[:, :-i]
            neg_score = score[:, i:]
            pos_score = pos_score.contiguous().view(-1)
            neg_score = neg_score.contiguous().view(-1)
            ones = torch.ones(pos_score.size()).cuda(score.device)
            loss_func = torch.nn.MarginRankingLoss(self.margin * i)
            TotalLoss += loss_func(pos_score, neg_score, ones)

        # gold summary loss
        pos_score = summary_score.unsqueeze(-1).expand_as(score)
        neg_score = score
        pos_score = pos_score.contiguous().view(-1)
        neg_score = neg_score.contiguous().view(-1)
        ones = torch.ones(pos_score.size()).cuda(score.device)
        loss_func = torch.nn.MarginRankingLoss(0.0)
        TotalLoss += loss_func(pos_score, neg_score, ones)
        
        return TotalLoss

In [ ]:
class rdass:
  def __init__(self,encoder,device):
    self.encoder = encoder
    for param in self.encoder.parameters():
        param.requires_grad = False
  
  def __call__(self, text_ids = None, label_ids = None, answer_ids = None):
    vector_text = self.encoder(text_ids).detach()['hidden_states'][-1][0,:] # vector_d
    vector_label = self.encoder(label_ids).detach()['hidden_states'][-1][0,:] # vector_r
    vector_answer = self.encoder(answer_ids).detach()['hidden_states'][-1][0,:] # vector_p

    return get_score(vector_text,vector_label,vector_answer)

### Model-Load & train code

- Encoder -> KoBART
- GLM 을 제외한 제일 성능 좋은 모델이고 한국어로 train이 되어 있어 선정함

In [ ]:
model = BartModel.from_pretrained(get_pytorch_kobart_model())
summary_model = MatchSum(encoder = model, candidate_num = 5,device = device, hidden_size=768) 

# model_for_eval = AutoModel.from_pretrained("klue/roberta-small")
# metric = rdass(model_for_eval,device)

N_EPOCHS = 3
optimizer = SGD(model.parameters(),lr =0.0001)
scheduler = CosineAnnealingWarmRestarts(optimizer,T_0 = len(train_dataloader)//10,T_mult = 500)
criterion = MarginRankingLoss(margin = 0.01)

In [ ]:
model.to(device)
summary_model.to(device)
wandb.init(project='summarization', entity='tkdlqh2')

for epoch in range(N_EPOCHS):
    
    print(f"*****Epoch {epoch} Train Start*****")
    print(f"*****Epoch {epoch} Total Step {len(train_dataloader)}*****")
    total_loss, batch_loss, batch_step = 0,0,0
    model.train()

    for step, batch in enumerate(train_dataloader):
        batch_step+=1
        text_input_ids = batch["text_input_ids"].to(device)        
        label_input_ids = batch["labels_input_ids"].to(device)

        model.zero_grad()
        optimizer.zero_grad()

        # forward
        output = summary_model.forward(text_input_ids, label_input_ids,batch["sentence_input_ids"])
        loss = criterion.get_loss(score = output["score"],summary_score = output["summary_score"])

        # loss 계산
        loss.backward()
        # optimizer 업데이트
        optimizer.step()
        # scheduler 업데이트
        scheduler.step()

        batch_loss += loss.item()
        total_loss += loss.item()

        learning_rate = optimizer.param_groups[0]['lr']
        wandb.log({'train/lr':learning_rate,"train/loss":loss.item()})

        if (step%50 == 0) and (step!=0):
            print(f"Step: {step} Loss: {batch_loss/batch_step:.4f} lr: {optimizer.param_groups[0]['lr']:.4f}")
            # 변수 초기화    
            batch_loss, batch_step = 0,0

    print(f"Epoch {epoch} Total Mean Loss : {total_loss/(step+1):.4f}")
    
    # with torch.no_grad():
    #   print('**Calculating validation results...**')
    #   total_metric, batch_step = 0,0
    #   model.eval()
    #   for step, batch in enumerate(train_dataloader):
    #       batch_step+=1
    #       text_input_ids = batch["text_input_ids"].to(device)        
    #       label_input_ids = batch["labels_input_ids"].to(device)

    #       # forward
    #       output = summary_model.forward(text_input_ids, label_input_ids,batch["sentence_input_ids"])
    #       metric_val = metric(text_input_ids,label_input_ids,output["golden_summary"])

    #       total_metric += metric_val.item()
    # print(f"Epoch {epoch} Total Mean Score : {total_metric/(step+1):.4f}")
    
    print(f"*****Epoch {epoch} Train Finished*****\n")
    torch.save(model.state_dict,f"/content/drive/MyDrive/NLP/kobart_model_{epoch}epoch.pth")